# Regional constraint vs gnomAD constraint
Exploratory data analysis comparing regional constraint annotations with gnomAD constraint data.

In [2]:
import pandas as pd
from src import constants as C

Get the gnomAD v4 constraint data

In [11]:
gn = pd.read_csv(
    C.GNOMAD_V4_CONSTRAINT,
    sep="\t",
    usecols=[
        "gene",
        "transcript",
        "mane_select",
        "lof.pLI",
        "lof.oe_ci.upper",
        "constraint_flags",
    ],
)
gn.head()

,gene,transcript,mane_select,lof.pLI,lof.oe_ci.upper,constraint_flags
0,A1BG,ENST00000263100,True,1.712900e-16,1.342,[]
1,A1BG,ENST00000600966,False,1.814400e-07,1.257,[]
2,A1BG,NM_130786.4,True,1.712900e-16,1.342,[]
3,A1CF,ENST00000282641,False,7.432800e-10,0.825,[]
4,A1CF,ENST00000373993,False,1.055900e-08,0.789,[]


Get our regional constraint data

reg = pd.read_csv("")